In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

In [2]:
sys.path.append('/Users/Elena/Desktop/TimeSeries/')

In [3]:
import time
import signal
import subprocess
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook')

//anaconda/envs/py35/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
from web.web_for_coverage import WebInterface
from timeseries.TimeSeries import TimeSeries

In [5]:

# when running from the terminal
# python go_server_persistent.py --ts_length 100 --db_name 'demo'

# here we load the server as a subprocess for demonstration purposes
server = subprocess.Popen(['python', '../go_server.py'])
time.sleep(1)

web = subprocess.Popen(['python', '../go_web.py'])
time.sleep(1)



In [6]:
web_interface = WebInterface()

In [8]:
results = web_interface.add_trigger(
            'junk', 'insert_ts', None, 'db:one:ts')
assert results[0] == 200
print(results)

(200, b'"Successfully added trigger"')


In [17]:
results = web_interface.add_trigger(
            'stats', 'insert_ts', ['mean', 'std'], None)
assert results[0] == 200
print(results[1])

b'"Successfully added trigger"'


In [13]:
def tsmaker(m, s, j):
    '''
    Helper function: randomly generates a time series for testing.

    Parameters
    ----------
    m : float
        Mean value for generating time series data
    s : float
        Standard deviation value for generating time series data
    j : float
        Quantifies the "jitter" to add to the time series data

    Returns
    -------
    A time series and associated meta data.
    '''

    # generate metadata
    meta = {}
    meta['order'] = int(np.random.choice(
        [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]))
    meta['blarg'] = int(np.random.choice([1, 2]))

    # generate time series data
    t = np.arange(0.0, 1.0, 0.01)
    v = norm.pdf(t, m, s) + j * np.random.randn(ts_length)

    # return time series and metadata
    return meta, TimeSeries(t, v)

In [32]:
mus = np.random.uniform(low=0.0, high=1.0, size=50)
sigs = np.random.uniform(low=0.05, high=0.4, size=50)
jits = np.random.uniform(low=0.05, high=0.2, size=50)
ts_length = 100

# initialize dictionaries for time series and their metadata
tsdict = {}
metadict = {}
for i, m, s, j in zip(range(50), mus, sigs, jits):
    meta, tsrs = tsmaker(m, s, j)
    # the primary key format is ts-1, ts-2, etc
    pk = "ts-{}".format(i)
    tsdict[pk] = tsrs
    meta['vp'] = False # augment metadata with a boolean asking if this is a  VP.
    metadict[pk] = meta


vpkeys = ["ts-{}".format(i) for i in np.random.choice(range(50), size=5, replace=False)]
for i in range(5):
    # add 5 triggers to upsert distances to these vantage points
    # data = json.dumps({'proc':'corr', 'onwhat':'insert_ts', 'target':["d_vp-{}".format(i)], 'arg':tsdict[vpkeys[i]].to_json()})
    # r = requests.post(self.web_url+'/add_trigger', data)

    r = web_interface.add_trigger('corr', 'insert_ts', ["d_vp-{}".format(i)], tsdict[vpkeys[i]].to_json())
    assert(r[0] == 200)
    # change the metadata for the vantage points to have meta['vp']=True
    metadict[vpkeys[i]]['vp'] = True

Having set up the triggers, now insert the time series, and upsert the metadata

==========================================

When it's first time to insert these keys in TSDB_server,
insert_ts will work and return TSDBStatus.OK

==========================================

In [19]:
for k in tsdict:
    results = web_interface.insert_ts(k, tsdict[k])
    assert results[0] == 200
    # upsert meta
    results = web_interface.upsert_meta(k, metadict[k])
    assert results[0] == 200


In [20]:
results = web_interface.add_trigger(
            'junk', 'insert_ts', None, 'db:one:ts')

In [21]:
results

(200, b'"Successfully added trigger"')

In [33]:
# ==========================================
# However if it's not first time to insert these keys,
# insert_ts will return TSDBStatus.INVALID_KEY
# ==========================================
# pick a random pk
idx = np.random.choice(list(tsdict.keys()))

# check that the time series is there now
results = web_interface.select({"primary_key": idx})
assert results[0] == 200


In [35]:
# delete an existing time series
results = web_interface.delete_ts(idx)
assert results[0] == 200


In [36]:
# check that the time series is no longer there
results = web_interface.select({"md":{"order": 1}, "fields":["ts"], "additional":{"sort_by":"-order"}})
assert results[0] == 200


In [27]:
# add the time series back in
results = web_interface.insert_ts(idx, tsdict[idx])
assert results[0] == 200